In [22]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

In [17]:
data = pd.read_csv('./../../communal/Ames_Housing_Price_Data_cleaned_2.csv')

In [18]:
data.head()
predict = data.SalePrice
log_predict = np.log(predict)
data.drop(['PID','Prop_Addr', 'mean_LotFrontage', 'lot_bucket','SalePrice','lat', 'long'], axis = 1, inplace = True)

In [19]:
data.head()

,GrLivArea,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,856,30,RL,64.9,7890,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,None,None,None,0,3,2010,WD,Normal
1,1049,120,RL,42.0,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal
2,1049,120,RL,42.0,4235,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,None,None,None,0,2,2009,WD,Normal
3,1001,30,C (all),60.0,6060,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,None,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,None,None,None,0,11,2007,WD,Normal
4,1039,70,RL,80.0,8146,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,None,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,None,None,None,0,5,2009,WD,Normal


In [20]:
dummy = pd.get_dummies(data, drop_first = True)

,GrLivArea,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_None,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSewr,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_None,BsmtQual_Po,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_None,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_Po,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageYrBlt_1900.0,GarageYrBlt_1906.0,GarageYrBlt_1908.0,GarageYrBlt_1910.0,GarageYrBlt_1914.0,GarageYrBlt_1915.0,GarageYrBlt_1916.0,GarageYrBlt_1917.0,GarageYrBlt_1918.0,GarageYrBlt_1920.0,GarageYrBlt_1921.0,GarageYrBlt_1922.0,GarageYrBlt_1923.0,GarageYrBlt_1924.0,GarageYrBlt_1925.0,GarageYrBlt_1926.0,GarageYrBlt_1927.0,GarageYrBlt_1928.0,GarageYrBlt_1929.0,GarageYrBlt_1930.0,GarageYrBlt_1931.0,GarageYrBlt_1932.0,GarageYrB

In [48]:
from collections import defaultdict
num_iters = 50

params = {'alpha' : np.linspace(1e-5,1e-3,100)}
lasso = Lasso(max_iter = 200000)
gs = GridSearchCV(lasso,param_grid=params, verbose = 3, cv = 5, n_jobs = -1)

d = defaultdict(int)
trainR2 = []
testR2 = []

for i in range(num_iters):
    print(i, end = '/r')
    np.random.seed(i)
    
    X_train, X_test, y_train, y_test = train_test_split(dummy,log_predict,test_size = .3)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    gs.fit(X_train ,y_train)
    best_estimator = gs.best_estimator_
    trainR2.append(best_estimator.score(X_train, y_train))
    testR2.append(best_estimator.score(X_test,y_test))
    
    
    important_features = pd.Series(best_estimator.coef_, 
                                  index = dummy.columns).sort_values(key = abs,ascending = False)

    
    for i, e in enumerate(important_features.index):
        d[e] += i
    
    
 
    
    
    
    

0/rFitting 5 folds for each of 100 candidates, totalling 500 fits
1/rFitting 5 folds for each of 100 candidates, totalling 500 fits
2/rFitting 5 folds for each of 100 candidates, totalling 500 fits
3/rFitting 5 folds for each of 100 candidates, totalling 500 fits
4/rFitting 5 folds for each of 100 candidates, totalling 500 fits
5/rFitting 5 folds for each of 100 candidates, totalling 500 fits
6/rFitting 5 folds for each of 100 candidates, totalling 500 fits
7/rFitting 5 folds for each of 100 candidates, totalling 500 fits
8/rFitting 5 folds for each of 100 candidates, totalling 500 fits
9/rFitting 5 folds for each of 100 candidates, totalling 500 fits
10/rFitting 5 folds for each of 100 candidates, totalling 500 fits
11/rFitting 5 folds for each of 100 candidates, totalling 500 fits
12/rFitting 5 folds for each of 100 candidates, totalling 500 fits
13/rFitting 5 folds for each of 100 candidates, totalling 500 fits
14/rFitting 5 folds for each of 100 candidates, totalling 500 fits
15/rF

In [52]:
final_d = {k: v/num_iters for k, v in d.items()}

In [49]:
print(trainR2)
print(testR2)

[0.9311895759777074, 0.9403874635128605, 0.9463017875842183, 0.917230566635031, 0.9450895180197965, 0.9433605210849304, 0.9342859410064115, 0.929855652481026, 0.9444712318955747, 0.9269912107939434, 0.9417909609915951, 0.9337664241499337, 0.9220526126458887, 0.9226912746807578, 0.9381890108491897, 0.9282833444877251, 0.9443055038893277, 0.9396141014245037, 0.9426332031708788, 0.9369545626229551, 0.9315961402283415, 0.9452461061704251, 0.9285955379235528, 0.9414503888384191, 0.9368974312415435, 0.9308136513619614, 0.9383410427611052, 0.934484717086082, 0.9251345068849115, 0.9395379601197654, 0.9473475859710874, 0.9217039140653321, 0.9350612368596185, 0.9301149848247326, 0.9298419182962085, 0.9298278172053467, 0.9342658683256431, 0.9149454693251516, 0.9343960346711205, 0.942180025705313, 0.9403303967568867, 0.9384685183424613, 0.9406177464549581, 0.9255889622404097, 0.9433769204983816, 0.9463685911758569, 0.9253543291751039, 0.9314135244453918, 0.9226398088630745, 0.9290456945965958]
[0.

In [50]:
import pprint

In [51]:
pprint.pprint(d)

defaultdict(<class 'int'>,
            {'1stFlrSF': 3220,
             '2ndFlrSF': 10483,
             '3SsnPorch': 13235,
             'Alley_None': 9819,
             'Alley_Pave': 11202,
             'BedroomAbvGr': 11562,
             'BldgType_2fmCon': 13057,
             'BldgType_Duplex': 1464,
             'BldgType_Twnhs': 786,
             'BldgType_TwnhsE': 3766,
             'BsmtCond_Fa': 7937,
             'BsmtCond_Gd': 4793,
             'BsmtCond_None': 16126,
             'BsmtCond_Po': 16544,
             'BsmtCond_TA': 15972,
             'BsmtExposure_Gd': 1679,
             'BsmtExposure_Mn': 4374,
             'BsmtExposure_No': 3194,
             'BsmtExposure_None': 8154,
             'BsmtFinSF1': 348,
             'BsmtFinSF2': 3383,
             'BsmtFinType1_BLQ': 11769,
             'BsmtFinType1_GLQ': 4687,
             'BsmtFinType1_LwQ': 3550,
             'BsmtFinType1_None': 16097,
             'BsmtFinType1_Rec': 10866,
             'BsmtFinType1_Unf

In [54]:
pprint.pprint(final_d)

{'1stFlrSF': 64.4,
 '2ndFlrSF': 209.66,
 '3SsnPorch': 264.7,
 'Alley_None': 196.38,
 'Alley_Pave': 224.04,
 'BedroomAbvGr': 231.24,
 'BldgType_2fmCon': 261.14,
 'BldgType_Duplex': 29.28,
 'BldgType_Twnhs': 15.72,
 'BldgType_TwnhsE': 75.32,
 'BsmtCond_Fa': 158.74,
 'BsmtCond_Gd': 95.86,
 'BsmtCond_None': 322.52,
 'BsmtCond_Po': 330.88,
 'BsmtCond_TA': 319.44,
 'BsmtExposure_Gd': 33.58,
 'BsmtExposure_Mn': 87.48,
 'BsmtExposure_No': 63.88,
 'BsmtExposure_None': 163.08,
 'BsmtFinSF1': 6.96,
 'BsmtFinSF2': 67.66,
 'BsmtFinType1_BLQ': 235.38,
 'BsmtFinType1_GLQ': 93.74,
 'BsmtFinType1_LwQ': 71.0,
 'BsmtFinType1_None': 321.94,
 'BsmtFinType1_Rec': 217.32,
 'BsmtFinType1_Unf': 122.3,
 'BsmtFinType2_BLQ': 199.96,
 'BsmtFinType2_GLQ': 120.02,
 'BsmtFinType2_LwQ': 298.3,
 'BsmtFinType2_None': 273.9,
 'BsmtFinType2_Rec': 289.76,
 'BsmtFinType2_Unf': 234.36,
 'BsmtFullBath': 26.86,
 'BsmtHalfBath': 253.88,
 'BsmtQual_Fa': 99.24,
 'BsmtQual_Gd': 101.58,
 'BsmtQual_None': 320.08,
 'BsmtQual_Po': 331

In [55]:
dict(sorted(final_d.items(), key=lambda item: item[1]))

{'GrLivArea': 0.02,
 'OverallQual': 1.04,
 'OverallCond': 2.38,
 'TotalBsmtSF': 3.78,
 'YearBuilt': 5.24,
 'BsmtFinSF1': 6.96,
 'GarageCars': 10.38,
 'Neighborhood_Crawfor': 11.62,
 'SaleCondition_Partial': 12.34,
 'Neighborhood_Somerst': 14.94,
 'BldgType_Twnhs': 15.72,
 'Neighborhood_MeadowV': 16.06,
 'Neighborhood_StoneBr': 17.58,
 'SaleCondition_Normal': 19.4,
 'ScreenPorch': 20.28,
 'CentralAir_Y': 20.58,
 'MSZoning_C (all)': 20.8,
 'GarageArea': 20.86,
 'Exterior1st_BrkFace': 22.36,
 'BsmtFullBath': 26.86,
 'Neighborhood_NridgHt': 27.3,
 'BldgType_Duplex': 29.28,
 'KitchenQual_TA': 30.16,
 'Functional_Typ': 32.34,
 'Condition1_Norm': 33.12,
 'YearRemodAdd': 33.42,
 'BsmtExposure_Gd': 33.58,
 'Neighborhood_ClearCr': 33.68,
 'KitchenQual_Fa': 34.54,
 'MSZoning_RL': 36.26,
 'Neighborhood_Edwards': 38.78,
 'GarageYrBlt_2008.0': 39.16,
 'PavedDrive_Y': 45.84,
 'KitchenQual_Gd': 49.04,
 'HeatingQC_TA': 49.4,
 'GarageCond_Fa': 50.02,
 'MSSubClass': 50.14,
 'Foundation_PConc': 50.22,
 'F